In [ ]:


#pip install geopy

In [1]:
import numpy as np
import pandas as pd
from geopy.distance import vincenty

In [3]:
data_trip = pd.read_csv('../data/2016_trip_data.csv',parse_dates=['starttime'])

In [4]:
data_trip.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear
0,431,2014-10-13 10:31:00,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1960.0
1,432,2014-10-13 10:32:00,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1970.0
2,433,2014-10-13 10:33:00,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1988.0
3,434,2014-10-13 10:34:00,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1977.0
4,435,2014-10-13 10:34:00,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1971.0


In [16]:
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(vincenty(newport_ri, cleveland_oh).miles)


538.390445157


In [21]:
station = pd.read_csv('../data/2016_station_data.csv')
station.head()

,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date
0,BT-01,3rd Ave & Broad St,47.618418,-122.350964,10/13/2014,18,NaN,18,NaN
1,BT-03,2nd Ave & Vine St,47.615829,-122.348564,10/13/2014,16,NaN,16,NaN
2,BT-04,6th Ave & Blanchard St,47.616094,-122.341102,10/13/2014,16,NaN,16,NaN
3,BT-05,2nd Ave & Blanchard St,47.613110,-122.344208,10/13/2014,14,NaN,14,NaN
4,CBD-03,7th Ave & Union St,47.610731,-122.332447,10/13/2014,20,NaN,20,NaN


In [23]:
data_trip['from_station_id'][1]


'CBD-06'

In [26]:
station.loc[2,'station_id']

'BT-04'

In [35]:
#station['station_id'].str.find('BT-03')==0

In [36]:
#index_temp = station.station_id[station.station_id == 'BT-03'].index.tolist()

In [205]:
def ride_distance(from_id,to_id):
    """calculate the distance between from_station and to_station"""
    
    #abnormal data
    if (data_trip.loc[i].to_station_id) == 'Pronto shop' : return 0
    if (data_trip.loc[i].to_station_id) == 'Pronto shop 2' : return 0
    if (data_trip.loc[i].to_station_id) == '8D OPS 02' : return 0
    if (data_trip.loc[i].from_station_id) == 'Pronto shop' : return 0
    #find the index of from_id in station list
    from_index = station.station_id[station.station_id == from_id].index.tolist()
    #get the latitude, longitude of from_sattion
    from_loc = (station.loc[from_index].lat.values[0], station.loc[from_index].long.values[0])
    
    to_index = station.station_id[station.station_id == to_id].index.tolist()
    to_loc = (station.loc[to_index].lat.values[0], station.loc[to_index].long.values[0])
    
    #measure the distance by Geopy
    distance = vincenty(from_loc, to_loc).miles
    
    return distance

In [38]:
station.loc[index_temp].lat

1    47.615829
Name: lat, dtype: float64

In [140]:
ride_distance(data_trip.loc[4].from_station_id,data_trip.loc[4].to_station_id)

0.38222522461707314

In [138]:
data_trip_dis = data_trip
data_trip_dis['trip_distance'] = 0

In [206]:
for i in range(227729,data_trip_dis.shape[0]):
    dis = ride_distance(data_trip.loc[i].from_station_id,data_trip.loc[i].to_station_id)
    data_trip_dis.set_value([i],'trip_distance',dis)

In [207]:
data_trip_dis.loc[167777]

trip_id                                           183100
starttime                            2016-02-10 17:18:00
stoptime                                 2/10/2016 17:31
bikeid                                          SEA00035
tripduration                                     757.539
from_station_name                  9th Ave N & Mercer St
to_station_name      Key Arena / 1st Ave N & Harrison St
from_station_id                                   DPD-01
to_station_id                                     SLU-19
usertype                                          Member
gender                                              Male
birthyear                                           1992
trip_distance                                   0.742477
Name: 167777, dtype: object

In [199]:
dis = ride_distance(data_trip.loc[161515].from_station_id,data_trip.loc[161515].to_station_id)
dis

0

In [200]:
data_trip_dis['trip_distance'][6]

0.19511052889118943

In [153]:
data_trip_dis.shape[0]

236065

In [203]:
i

227729

In [183]:
data_trip.loc[i].to_station_id

'Pronto shop 2'

In [221]:
#average speed   (m/s)
data_trip_dis['average_speed'] = data_trip_dis.trip_distance*1609.34/data_trip_dis.tripduration
data_trip_dis.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear,trip_distance,average_speed
0,431,2014-10-13 10:31:00,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1960.0,0.382225,0.623906
1,432,2014-10-13 10:32:00,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1970.0,0.382225,0.664019
2,433,2014-10-13 10:33:00,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1988.0,0.382225,0.695982
3,434,2014-10-13 10:34:00,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1977.0,0.382225,0.710364
4,435,2014-10-13 10:34:00,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1971.0,0.382225,0.665781


In [222]:
times = pd.DatetimeIndex(data_trip_dis['starttime'])
times.date

array([datetime.date(2014, 10, 13), datetime.date(2014, 10, 13),
       datetime.date(2014, 10, 13), ..., datetime.date(2016, 8, 31),
       datetime.date(2016, 8, 31), datetime.date(2016, 8, 31)], dtype=object)

In [224]:
#time
data_trip_dis['date'] = times.date
data_trip_dis['hour'] = times.hour
data_trip_dis.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear,trip_distance,average_speed,date,hour
0,431,2014-10-13 10:31:00,10/13/2014 10:48,SEA00298,985.935,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1960.0,0.382225,0.623906,2014-10-13,10
1,432,2014-10-13 10:32:00,10/13/2014 10:48,SEA00195,926.375,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1970.0,0.382225,0.664019,2014-10-13,10
2,433,2014-10-13 10:33:00,10/13/2014 10:48,SEA00486,883.831,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1988.0,0.382225,0.695982,2014-10-13,10
3,434,2014-10-13 10:34:00,10/13/2014 10:48,SEA00333,865.937,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Female,1977.0,0.382225,0.710364,2014-10-13,10
4,435,2014-10-13 10:34:00,10/13/2014 10:49,SEA00202,923.923,2nd Ave & Spring St,Occidental Park / Occidental Ave S & S Washing...,CBD-06,PS-04,Member,Male,1971.0,0.382225,0.665781,2014-10-13,10


In [226]:
#save data
data_trip_dis.to_csv('../data/2016_trip_data_update.csv')